In [1]:
import torch
from torch import nn
from torch.nn import functional as F

### Importing dependencies and downloading pre-trained model

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "bigscience/bloom-560m"
#peft_model_id = "./checkpoint/BLOOM-560m-LoRA"
model_id = "facebook/opt-350m"
peft_model_id = "./checkpoint/opt-350m-lora"

#loading model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # torch_dtype=torch.float32,
    device_map='auto',
)

#loading tokenizer for this model (which turns text into an input for the model)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/simon/anaconda3/envs/wheels/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define LoRA linear module

In [3]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class LoraConfig:
    r: int = 8
    lora_alpha: int = 8
    lora_dropout: float = 0.1
    target_modules = ["q_proj", "v_proj"]

class LoraLinear(nn.Module):
    def __init__(
        self,        
        pre_model: nn.Linear,
        config: LoraConfig,
    ):
        super().__init__()
        
        self.pre_model = pre_model
        self.config = config
        
        self.lora_dropout = nn.Dropout(config.lora_dropout)
        
        self.lora_A = nn.Linear(pre_model.in_features, config.r, bias=False)
        nn.init.normal_(self.lora_A.weight)
        self.lora_B = nn.Linear(config.r, pre_model.out_features, bias=False)        
        nn.init.constant_(self.lora_B.weight, 0)

        self.scaling = config.lora_alpha / config.r
        self.lora_enabled = True
    

    def enable_lora(self) -> None:
        self.lora_enabled = True    
    
    def disable_lora(self) -> None:
        self.lora_enabled = False

    def forward(self, input: torch.Tensor):
        out = self.pre_model(input)
        if self.lora_enabled:
            out += self.lora_dropout(self.lora_B(self.lora_A(input))) * self.scaling

        return out

class LoraAdapter(nn.Module):
    """
        pt_model: Pretrained model
        config: Lora configuration
    """
    def __init__(
        self,        
        pt_model: nn.Module,
        config: LoraConfig,
        enable_lora: bool = True,
    ):
        super().__init__()        
        self.lora_modules = nn.ModuleList()
        self.pt_model = pt_model
        self.config = config                
        
        self.lora_enabled = enable_lora        
        if enable_lora:
            self._apply_lora_linear(pt_model)

    def _apply_lora_linear(self, model: nn.Module) -> None:
        for attr_name, children in model.named_children():
            for para in children.parameters():
                para.requires_grad = False
                
            if attr_name in self.config.target_modules and isinstance(children, nn.Linear):
                lora_linear = LoraLinear(children, self.config)
                setattr(model, attr_name, lora_linear)
                self.lora_modules.append(lora_linear)
            else:
                self._apply_lora_linear(children)
    
    def disable_lora(self) -> None:
        self.lora_enabled = False
        for module in self.lora_modules:
            module.disable_lora()
    
    def enable_lora(self) -> None:
        self.lora_enabled = True
        for module in self.lora_modules:
            module.enable_lora()
                
    # def forward(self, input: torch.Tensor):
    #     return self.pt_model(input)

### Applying LoRA on model

In [4]:
config = LoraConfig()
lora_adapter = LoraAdapter(model, config)

### Comparing parameters before and after LoRA

In [5]:
def compare_parameters(model):
    trainable_params = 0
    all_param = 0

    #iterating over all parameters
    for _, param in model.named_parameters():
        #adding parameters to total
        all_param += param.numel()
        #adding parameters to trainable if they require a graident
        if param.requires_grad:
            trainable_params += param.numel()

    #printing results
    print(f"trainable params: {trainable_params:,}")
    print(f"all params: {all_param:,}")
    print(f"trainable: {100 * trainable_params / all_param:.2f}%")

compare_parameters(lora_adapter.pt_model)

trainable params: 786,432
all params: 331,982,848
trainable: 0.24%


### Loading and reformating SQUAD dataset

In [6]:
from datasets import load_dataset
qa_dataset = load_dataset("squad_v2")

In [9]:
len(qa_dataset['train']),len(qa_dataset['validation'])

(130319, 11873)

In [10]:
def create_prompt(context, question, answer):
  if len(answer["text"]) < 1:
    answer = "Cannot Find Answer"
  else:
    answer = answer["text"][0]
  prompt_template = f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:\n{answer}</s>"
  return prompt_template

#applying the reformatting function to the entire dataset
mapped_qa_dataset = qa_dataset.map(lambda samples: tokenizer(create_prompt(samples['context'], samples['question'], samples['answers'])))

### Fine tuning

This code is largly co-opted. In the absence of a rigid validation
procedure, the best practice is to just copy a successful tutorial or,
better yet, directly from the documentation.

In [11]:
import transformers

trainer = transformers.Trainer(
    model = lora_adapter.pt_model,
    train_dataset = mapped_qa_dataset["train"],
    eval_dataset = mapped_qa_dataset["validation"],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        num_train_epochs = 1,
        learning_rate = 5e-4,        
        lr_scheduler_type = "cosine",
        logging_steps = 100,
        eval_steps = 100,
        evaluation_strategy = "steps",
        output_dir = 'results',
        report_to = "tensorboard",
    ),
    # args = transformers.TrainingArguments(
    #     output_dir="results",
    #     per_device_train_batch_size=4,
    #     per_device_eval_batch_size=4,
    #     evaluation_strategy="steps",
    #     eval_steps=50,
    #     logging_steps=50,
    #     gradient_accumulation_steps=1,
    #     num_train_epochs=1,
    #     weight_decay=0.1,
    #     warmup_steps=10,
    #     lr_scheduler_type="cosine",
    #     learning_rate=5e-4,
    #     # fp16=True,
    #     report_to="tensorboard",
    # ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# model.save_pretrained(peft_model_id)

/home/simon/anaconda3/envs/wheels/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/simon/anaconda3/envs/wheels/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,3.124500,3.093397
20,3.189600,3.072169
30,3.186400,3.024478


### Test

In [ ]:
from IPython.display import display, Markdown

def generate(model, tokenizer, context, question):
    batch = tokenizer(f"**CONTEXT:**\n{context}\n\n**QUESTION:**\n{question}\n\n**ANSWER:**\n", return_tensors='pt', return_token_type_ids=False)
    # batch = batch.to(device='cuda')

    #raw model
    return tokenizer.decode(model.generate(**batch, max_new_tokens=200)[0], skip_special_tokens=True)

def compare_inference(adapter, tokenizer, context, question):
    adapter.enable_lora()
    out = generate(adapter.pt_model, tokenizer, context, question)
    display(Markdown("# Finetuned Model\n"))
    display(Markdown(out))
    adapter.disable_lora()
    out = generate(adapter.pt_model, tokenizer, context, question)
    display(Markdown("# Raw Model\n"))
    display(Markdown(out))
    adapter.enable_lora()

In [ ]:
context = "You are a monster, and you eat yellow legos."
question = "What is the best food?"

compare_inference(lora_adapter, tokenizer, context, question)

In [ ]:
context = "you are a math wizard"
question = "what is 1+1 equal to?"

compare_inference(lora_adapter, tokenizer, context, question)

In [ ]:
context = "Answer the riddle"
question = "What gets bigger the more you take away?"

compare_inference(lora_adapter, tokenizer, context, question)